In [8]:
%reset -f
import numpy as np
from scipy.stats import geom
import sys
from datetime import datetime
import os

In [9]:
output_log_file = "experiment_4_H.log"
log_file = open(output_log_file, 'a')
sys.stdout = log_file
print(output_log_file)
print(datetime.now().strftime("%a %b %d %H:%M:%S %Y"))

In [10]:
DEFAULT_WEIGHT_VALUE = 0  
image_mode = False
debug_mode = False

%run attack_graph_MIR100.ipynb

In [11]:
defense_rate_list = [1, 2, 3]

In [12]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    """
    Calculates probabilities for attacker movement along route.
    Returns probability distribution over possible ending nodes.
    """

    # Part 1: Extract hardness values from all edges and append them
    # into one numpy array
    # Calculate hardness values for each edge 
    hardness = []
    for i in range(len(route) - 1):
        start_node = route[i]
        end_node = route[i + 1]
        
        # Initialize variables for max weight loop
        weights = []
        # Collect all weights for max
        for edge in graph[start_node][end_node].values():
            weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get maximum weight
        max_weight = max(weights) if weights else DEFAULT_WEIGHT_VALUE
        
        # Initialize variables for min weight loop
        min_weights = []
        # Collect all weights for min
        for edge in graph[start_node][end_node].values():
            min_weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get minimum weight
        min_weight = min(min_weights) if min_weights else DEFAULT_WEIGHT_VALUE
            
        # Convert weights to probabilities
        # We could take max_weight or min_weight here
        # hardness.append(np.exp(-max_weight))

        # Important: We use min_weight here because of the following reason:
        # Since the formula to calculate hardness in R is hardness = exp(-weight)
        # taking the minimum weight will give us the maximum hardness
        # which translates to the path being EASIEST to traverse.
        # Yes hardness of 1 means path is trivial, hardness 0 means path is impossible
        hardness.append(np.exp(-min_weight))

    
    # Convert to arrays
    hardness = np.array(hardness)

    # print(f'Hardness: {hardness}')
    
    # Calculate attack rate using geometric mean if not provided
    if attack_rate is None or attack_rate == 0:
        # Geometric mean function (mirrors the R geomean function)
        def geom_mean(x):
            # I filter out zeros to avoid log(0)
            positive_values = x[x > 0]
            if len(positive_values) == 0:
                return 1  # Default if all values are zero or array is empty
            return np.exp(np.mean(np.log(positive_values)))
        
        attack_rate = 1 / geom_mean(hardness)
    
    # Calculate probability distribution using geometric distribution
    # Equivalent to R's dgeom function
    prob = attack_rate / (attack_rate + defense_rate)
    
    # Create range of values from 0 to length(route)-1
    x_values = np.arange(len(route))
    
    # Calculate probability mass function (pmf) for geometric distribution
    # pdf_d = geom.pmf(x_values, prob)
    pdf_d = prob * (1-prob)**x_values
    
    # Normalize to ensure probabilities sum to 1
    pdf_d = pdf_d / np.sum(pdf_d)
    
    # print(f"This is the final pdf that is returned in the end: {pdf_d}")
    return pdf_d

# Example usage:
# route = [1, 2, 3, 4]
# defense_rate = 2
# pdf = random_steps(route, attack_graph=G, defense_rate=defense_rate)

In [13]:
%run heuristic_defense.ipynb

In [14]:
sys.stdout = sys.__stdout__
log_file.close()
with open(output_log_file, 'r') as f:
    print(f.read())